# Surrogate Modeling Regression Example
This example illustrates the difference between Kriging and the (Batch) XGBoost algorithm. 
The aim is to show the difference between Kriging and XGBoost on the exact same examples in
the standard setting of regression. We do not expect a great difference in performance.
Note that there is no iterative training in this example. 

## Setup the Environment
Note that this code has only been tested on Python 3

In [1]:
%config InlineBackend.figure_format = 'retina'
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
""" Ignore Warnings """
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [3]:
from functions import *
import matplotlib

In [4]:
matplotlib.use('Agg')

### Initialize the parameters and constants

In [5]:
# Set the ABM Evaluation Budget
budget = 50

# Set out-of-sample test and montecarlo sizes
test_size = 100
montecarlos = 100

# Get an on out-of-sample test set that does not have combinations from the
# batch or iterative experiments
final_test_size = (test_size * montecarlos)

# Set the ABM parameters and support
islands_exploration_range = np.array([
    (0.0, 10),  # rho
    (0.8, 2.0),  # alpha
    (0.0, 1.0),  # phi
    (0.0, 1.0),  # pi
    (0.0, 1.0)])  # eps

param_dims = islands_exploration_range.shape[0]

## Evaluate the entire Budget in Batch
Draw these samples according to a pseudo-random, well spaced generator. Here we use Sobol sequences because of their known coverage and compute performance.

In [ ]:
n_dimensions = islands_exploration_range.shape[0]
evaluated_set_X_batch = get_sobol_samples(n_dimensions, budget, islands_exploration_range)
evaluated_set_y_batch = evaluate_islands_on_set(evaluated_set_X_batch)

/home/amir/PycharmProjects/calib_ex/functions.py:236: RuntimeWarning: divide by zero encountered in log
  log_GDP = np.log(GDP)


## Generate unique out-of-sample test set
Note that this is taken uniformly at random from the support. If you take points that are structured by the sobol sequence generator, then the performance evaluation will not be blind, but biased. Given that the Sobol sequence is generated in an arguably linear fashion, this favors Kriging.

In [ ]:
oos_set = get_unirand_samples(n_dimensions, final_test_size*budget, islands_exploration_range)

selections = []
for i, v in enumerate(oos_set):
    if (v not in evaluated_set_X_batch):
        selections.append(i)
oos_set = unique_rows(oos_set[selections])[:final_test_size]

# Evaluate the test set for the ABM response
y_test = evaluate_islands_on_set(oos_set)

## Compute the Kriging Surrogate

In [ ]:
# We use the default Gaussian Kernel and L-BFGS optimizer.
surrogate_models_kriging = GaussianProcessRegressor(random_state=0)
surrogate_models_kriging.fit(evaluated_set_X_batch, evaluated_set_y_batch)

## Compute the XGBoost Surrogate

In [ ]:
# This surrogate will not have multiple iterations. It will run on the entire budget of evaluations.
surrogate_model_XGBoost = fit_surrogate_model(evaluated_set_X_batch, evaluated_set_y_batch)

# Evaluate both surrogates on the test set

In [ ]:
y_hat_test = [None] * 2
mse_perf = np.zeros((2, montecarlos))

y_hat_test[0] = surrogate_models_kriging.predict(oos_set)
y_hat_test[1] = surrogate_model_XGBoost.predict(oos_set)

# MSE performance
for sur_idx in range(len(y_hat_test)):
    for i in range(montecarlos):
        mse_perf[sur_idx, i] = mean_squared_error(y_test[i * test_size:(i + 1) * test_size],
                                                  y_hat_test[int(sur_idx)][i * test_size:(i + 1) * test_size])

## Plot the out-of-sample Monte Carlo performance densities for each of the methods

In [ ]:
experiment_labels = ["Kriging", "XGBoost (Batch)"]

mse_perf = pd.DataFrame(mse_perf, index=experiment_labels)
ax = plt.axes()
sns.heatmap(mse_perf, ax=ax)
ax.set_title('MSE Performance Heatmap over Monte-Carlos')
ax.set_xlabel('')

fig, ax = plt.subplots(figsize=(12, 5))

k_label = "Kriging: Mean " + str(mse_perf.iloc[0, :].mean()) + ", Variance " + str(mse_perf.iloc[0, :].var())
xgb_label = "XGBoost: Mean " + str(mse_perf.iloc[1, :].mean()) + ", Variance " + str(mse_perf.iloc[1, :].var())

fig1 = sns.distplot(mse_perf.iloc[0, :], label=k_label, ax=ax)
fig2 = sns.distplot(mse_perf.iloc[1, :], label=xgb_label, ax=ax)

plt.title("Out-Of-Sample Prediction Performance")
plt.xlabel('Mean-Squared Error')
plt.yticks(fig1.get_yticks(), fig1.get_yticks() / 100)
plt.ylabel('Density')
plt.legend()
fig.savefig("xgboost_kriging_ba_comparison_" + str(budget) + ".png");